In [1]:
# !pip install -r requirements.txt
## Base of notebooks
%load_ext autoreload
%autoreload 2
import tensorflow as tf

In [2]:
from datetime import datetime

now = datetime.now() # current date and time
date = now.strftime("%Y-%m-%d")
img_size = 128
task  = 'multiclass-200mg'

In [3]:
import sys 
sys.path.append('..')

from utils import *
import models

In [4]:
base_data = stack_images(im_size=img_size, directory= "C:\\Users\\baran.tanriverdi\\Final Data")

100%|██████████| 43/43 [00:06<00:00,  6.44it/s]


Al for 5 Cannot found!


100%|██████████| 150/150 [00:23<00:00,  6.49it/s]

Fe2 : 51
Fe5 : 49
Fe10 : 100
Fe20 : 161
Fe200 : 153
Al2 : 43
Al10 : 16
Al20 : 150
Al200 : 136
Pb2 : 53
Pb5 : 50
Pb10 : 97
Pb20 : 150
Pb200 : 150
dict_keys(['Fe2', 'Fe5', 'Fe10', 'Fe20', 'Fe200', 'Al2', 'Al10', 'Al20', 'Al200', 'Pb2', 'Pb5', 'Pb10', 'Pb20', 'Pb200'])


In [5]:
np.asarray(base_data['Fe5']).shape

(49, 128, 128, 3)

In [6]:
# Configyure for current experiment

x1 = []
for imset in [base_data['Fe200']]:
    for im in imset:
        x1.append(im)
print(np.asarray(x1).shape)

x2 = []
for imset in [base_data['Al200']]:
    for im in imset:
        x2.append(im)
print(np.asarray(x2).shape)
        
x3 = []
for imset in [base_data['Pb200']]:
    for im in imset:
        x3.append(im)
print(np.asarray(x3).shape)

imlist = []
one_hot_vector = np.eye(3)
for x in x1:
    imlist.append([x, one_hot_vector[0]])
for x in x2:
    imlist.append([x, one_hot_vector[1]])  
for x in x3:
    imlist.append([x, one_hot_vector[2]]) 
    
print(len(imlist), len(x1), len(x2), len(x3))

data = np.asarray(imlist)

np.random.shuffle(data)

x = np.asarray([i[0] / 255.0 for i in data])
y = np.asarray([i[1] for i in data])

print(len(x), len(y))
boundary = int(len(x) * 0.75)

train_x = x[:boundary]
train_y = y[:boundary]
test_x = x[boundary:]
test_y = y[boundary:]

print(len(train_x), len(test_x))

(153, 128, 128, 3)
(136, 128, 128, 3)
(150, 128, 128, 3)
439 153 136 150
439 439
329 110


C:\Users\baran.tanriverdi\AppData\Local\Temp\ipykernel_7740\1130292994.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.asarray(imlist)


In [8]:
histories = []
models_ = [
    models.unet(input_shape=(img_size, img_size, 3), number_of_classes=3), 
    models.vgg5_block(input_shape=(img_size, img_size, 3), number_of_classes=3),
    #models.vgg16(input_shape=(img_size, img_size, 3), number_of_classes=3), 
    #models.vgg19(input_shape=(img_size, img_size, 3), number_of_classes=3),
    #models.resnet50(input_shape=(img_size, img_size, 3), number_of_classes=3), 
    #models.deeplabv3plus(input_shape=(img_size, img_size, 3), number_of_classes=3),
    #models.nasnet_mobile(input_shape=(img_size, img_size, 3), number_of_classes=3), 
    #models.inception_resnetv2(input_shape=(img_size, img_size, 3), number_of_classes=3), 
    #models.double_unet(input_shape=(img_size, img_size, 3), number_of_classes=3)
]

for model in tqdm(models_):
    print(model.name)
    history =model.fit(train_x, train_y, batch_size=10,
                    validation_split=0.2,
                    epochs=25, 
                    #callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=2,),
                    verbose = 1)
    histories.append(history)
    MODEL_NAME = f"{date}_{model.name}_{img_size}_{task}"
    save_logs(history, MODEL_NAME)
        #save m-odel
    tf.keras.models.save_model(
        model,
        f'./models/{task}/{MODEL_NAME}.h5')
           

Model: "vgg5_block"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_34 (Conv2D)          (None, 128, 128, 32)      896       
                                                                 
 conv2d_35 (Conv2D)          (None, 128, 128, 32)      9248      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_36 (Conv2D)          (None, 64, 64, 64)        18496     
                                                                 
 conv2d_37 (Conv2D)          (None, 64, 64, 64)        36928     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 32, 32, 64)       0         
 2D)                                                    

  0%|          | 0/2 [00:00<?, ?it/s]

unet
Epoch 1/25
27/27 [==============================] - 1386s 50s/step - loss: 1391.8234 - accuracy: 0.3916 - f1_score: 0.1883 - cohen_kappa: 0.0775 - MatthewsCorrelationCoefficient: 0.0778 - auc: 0.5393 - val_loss: 1.0863 - val_accuracy: 0.3333 - val_f1_score: 0.0000e+00 - val_cohen_kappa: 0.1076 - val_MatthewsCorrelationCoefficient: 0.2169 - val_auc: 0.5765
Epoch 2/25
27/27 [==============================] - 1331s 49s/step - loss: 1.6161 - accuracy: 0.4259 - f1_score: 0.1909 - cohen_kappa: 0.1164 - MatthewsCorrelationCoefficient: 0.1227 - auc: 0.5830 - val_loss: 1.1417 - val_accuracy: 0.2273 - val_f1_score: 0.0874 - val_cohen_kappa: -0.0042 - val_MatthewsCorrelationCoefficient: -0.0165 - val_auc: 0.4678
Epoch 3/25
27/27 [==============================] - 1318s 49s/step - loss: 1.1514 - accuracy: 0.3802 - f1_score: 0.0661 - cohen_kappa: 0.0716 - MatthewsCorrelationCoefficient: 0.0761 - auc: 0.5569 - val_loss: 1.0973 - val_accuracy: 0.3030 - val_f1_score: 0.0000e+00 - val_cohen_kappa:

27/27 [==============================] - 1337s 50s/step - loss: 0.4467 - accuracy: 0.8441 - f1_score: 0.8267 - cohen_kappa: 0.7644 - MatthewsCorrelationCoefficient: 0.7653 - auc: 0.9481 - val_loss: 0.9009 - val_accuracy: 0.7424 - val_f1_score: 0.7328 - val_cohen_kappa: 0.6060 - val_MatthewsCorrelationCoefficient: 0.6086 - val_auc: 0.8837
Epoch 25/25
27/27 [==============================] - 1330s 49s/step - loss: 0.3769 - accuracy: 0.8479 - f1_score: 0.8430 - cohen_kappa: 0.7703 - MatthewsCorrelationCoefficient: 0.7724 - auc: 0.9620 - val_loss: 0.9133 - val_accuracy: 0.7727 - val_f1_score: 0.7682 - val_cohen_kappa: 0.6463 - val_MatthewsCorrelationCoefficient: 0.6479 - val_auc: 0.9031


 50%|█████     | 1/2 [9:15:05<9:15:05, 33305.56s/it]

vgg5_block
Epoch 1/25
27/27 [==============================] - 112s 4s/step - loss: 1.2765 - accuracy: 0.3878 - f1_score: 0.3236 - auc: 0.5501 - val_loss: 1.0185 - val_accuracy: 0.5000 - val_f1_score: 0.0000e+00 - val_auc: 0.7121
Epoch 2/25
27/27 [==============================] - 96s 4s/step - loss: 1.0862 - accuracy: 0.4259 - f1_score: 0.2621 - auc: 0.5975 - val_loss: 0.9988 - val_accuracy: 0.5152 - val_f1_score: 0.1285 - val_auc: 0.7625
Epoch 3/25
27/27 [==============================] - 97s 4s/step - loss: 0.9820 - accuracy: 0.5095 - f1_score: 0.3464 - auc: 0.7047 - val_loss: 1.0446 - val_accuracy: 0.4394 - val_f1_score: 0.2516 - val_auc: 0.6585
Epoch 4/25
27/27 [==============================] - 97s 4s/step - loss: 0.9459 - accuracy: 0.5209 - f1_score: 0.3947 - auc: 0.7266 - val_loss: 0.8124 - val_accuracy: 0.6212 - val_f1_score: 0.5268 - val_auc: 0.8262
Epoch 5/25
27/27 [==============================] - 96s 4s/step - loss: 0.8828 - accuracy: 0.5932 - f1_score: 0.4712 - auc: 0.77

100%|██████████| 2/2 [9:55:04<00:00, 17852.50s/it]  
